UsageError: %%time is a cell magic, but the cell body is empty. Did you mean the line magic %time (single %)?


In [189]:
from selenium import webdriver
from time import sleep
import json
import re as regx
import os

import face_recognition # https://github.com/ageitgey/face_recognition
from PIL import Image # https://pillow.readthedocs.io/en/stable/

In [208]:
__TESTING_FILEPATH__ = "International Space Station/Expedition 1"
__TESTING_DIRECTORY__ = "International Space Station/"

In [41]:
lastNames = open("lastNames.txt","r")
    
data = {}
data['names'] = []

for fullName in lastNames:
    data['names'].append({
    'fname': regx.search("\w+(?= )", fullName)[0],
    'lname': regx.search("(?<= )[\w-]+[^.\n]", fullName)[0]
    })
    
with open('lastNames.json', 'w') as outfile:
    json.dump(data, outfile)

In [57]:
with open('lastNames.json', 'r') as f:
    lastNameDict = json.load(f)

lastNameDict["names"][0]

{'fname': 'joseph', 'lname': 'acaba'}

In [236]:
def findResults(source):
    people = []
    for i in lastNameDict["names"]:
        re = "({0}) (\w.)*\ ?({1})".format(i["fname"], i["lname"])
        result = regx.search(re, source, regx.IGNORECASE)
        if result is not None:
            people.append(result[0])
        
    re = '|'.join("{}".format(i) for i in people)
    result = regx.findall(re, source)
    
    if regx.search("right to left", source) is not None:
        result.reverse()
    
    return result
     
def readTextFile(fp):
    with open("{}.txt".format(fp), 'r') as file:
        data = file.read()
    return data

def transformCoord(coordinates):
    return (coordinates[3], coordinates[0], coordinates[1], coordinates[2])

def saveFaces(fp, output = "./temp"):
    try:
        os.mkdir(output)
    except:
        print("Path already exists")
    
    imageDesc = readTextFile(fp)
    people = findResults(imageDesc)
    
    photoName = fp.replace(" ", "_")
    photoName = photoName[photoName.find("/"):]
    
    sourePhoto = "{}.jpg".format(fp)
    image = face_recognition.load_image_file(sourePhoto)
    face_locations = face_recognition.face_locations(image)
    
    print("{0}: Found {1} people in the photo and {2} in the description".format(fp, len(face_locations), len(people)))
    
    savedPhotos = {}
    
    for i in range(0, len(face_locations)):
        try:
            tempPhotoName = "{0}{1}-{2}.jpg".format(output, photoName, people[i].replace(" ", "_"))
            savedPhotos[tempPhotoName] = Image.open(sourePhoto).crop(transformCoord(face_locations[i]))
        except:
            print("There was an error saving a file")
            Image.open(sourePhoto).crop(transformCoord(face_locations[i])).show()
    if len(savedPhotos) == len(face_locations):
        print(savedPhotos)
        for name, photo in savedPhotos.items():
            photo.save(name)

In [177]:
findResults(readTextFile(__TESTING_FILEPATH__))

['Yuri Gidzenko', 'Sergei Krikalev', 'William M. Shepherd']

In [204]:
saveFaces(__TESTING_FILEPATH__)

Path already exists
There was an error saving a file


In [237]:
for filename in os.listdir(__TESTING_DIRECTORY__):
    if filename.endswith(".jpg"): 
        saveFaces(__TESTING_DIRECTORY__+filename[:-4])
        

Path already exists
International Space Station/TMA-08M Landing: Found 11 people in the photo and 1 in the description
There was an error saving a file
There was an error saving a file
There was an error saving a file
There was an error saving a file
There was an error saving a file
There was an error saving a file
There was an error saving a file
There was an error saving a file
There was an error saving a file
There was an error saving a file
Path already exists
International Space Station/Zvezda Launch: Found 0 people in the photo and 931 in the description
{}
Path already exists
International Space Station/New York City on September 11, 2001: Found 0 people in the photo and 1 in the description
{}
Path already exists
International Space Station/Ron Garan: Found 0 people in the photo and 249 in the description
{}
Path already exists
International Space Station/STS-111 Landing: Found 0 people in the photo and 242 in the description
{}
Path already exists
International Space Station/P

International Space Station/X-38 on B-52 Wing Pylon: Found 0 people in the photo and 481 in the description
{}
Path already exists
International Space Station/SpaceX CRS5 Launch: Found 0 people in the photo and 527 in the description
{}
Path already exists
International Space Station/X-38 Ship 2 in Free Flight: Found 0 people in the photo and 1241 in the description
{}
Path already exists
International Space Station/Soyuz TMA-17 Lands: Found 0 people in the photo and 2 in the description
{}
Path already exists
International Space Station/Randy Bresnik: Found 0 people in the photo and 234 in the description
{}
Path already exists
International Space Station/Expedition 2 Crew: Found 4 people in the photo and 10 in the description
{'./temp/Expedition_2_Crew-Kent_V._Rominger.jpg': <PIL.Image.Image image mode=RGB size=155x155 at 0x113BFBA90>, './temp/Expedition_2_Crew-Yuri_V._Lonchakov.jpg': <PIL.Image.Image image mode=RGB size=185x186 at 0x113BFB668>, './temp/Expedition_2_Crew-Yury_V._Usac